In [122]:
import os
os.chdir('/home/composersyf/Documents/Google Civic Information API/WA Addresses/First_Batch')

In [123]:
import glob
county_list=glob.glob('*.csv')

In [124]:
import pandas as pd
import numpy as np

In [125]:
def str_to_int(text):
    try:
        n=int(text)
        return str(int(text))
    except ValueError:
        return None
    
def is_not_None(entry):
    if entry is None:
        return False
    else:
        return True

def zipcode(code):
    try:
        s=int(str(code)[:5])
        return str(code)[:5]
    except ValueError:
        return ''

def generate_address(ndarr,county_name):
    if ndarr[2]=='':
        return np.hstack((ndarr[:2],[county_name,"WA"],ndarr[3]))
    elif ndarr[3]=='':
        return np.hstack((ndarr[:3],"WA"))
    else:
        return np.hstack((ndarr[:3],"WA",ndarr[3]))

In [126]:
sample_address_df=[]
for county in county_list:
    county_name=county[:-4]
    first_df=pd.read_csv(county)
    first_df_1=first_df[~pd.isnull(np.array(first_df.NUMBER)) & ~pd.isnull(np.array(first_df.STREET))]
    first_df_1['NUMBER']=first_df_1['NUMBER'].apply(str_to_int)
    first_df_2=first_df_1[np.array(first_df_1['NUMBER'].apply(is_not_None))]
    first_df_3=first_df_2[['NUMBER','STREET','CITY','POSTCODE']]
    first_df_3['POSTCODE']=first_df_3['POSTCODE'].apply(zipcode)
    first_df_4=first_df_3[~np.array(pd.isnull(first_df_3['CITY'])) | ~np.array(first_df_3.POSTCODE=='')]
    first_df_4=first_df_4.fillna('')
    combined_address=[None]*first_df_4.shape[0]
    for j in range(first_df_4.shape[0]):
        address_arr=generate_address(np.array(first_df_4.iloc[j,:]),county_name)
        combined_address[j]=" ".join(address_arr)
    address_sample=np.random.choice(combined_address,size=min(200,len(combined_address)))
    county_column=[county_name]*len(address_sample)
    sample_address_df.append(pd.DataFrame({"county":county_column,"address":address_sample}))

/home/composersyf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/composersyf/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [135]:
final_df=pd.concat(sample_address_df,axis=0,join="outer").reset_index([0]).drop('index',axis=1)

In [136]:
final_df.to_csv("../sample_WA_addresses.csv",header=False,index=False)